This notebook processes the raw NOAA data found at https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/.
We only want stations that appear in our data and we do not care about particular time stamps of the readings. Finally, we will have to pivot our TYPE column so that we can extract PRCP, TMAX, TMIN and SNOW separately. When reading these .csvs the code will have to ignore the first two header lines. THe output headers are "STATIONID", "DATE" (YYYYMMDD), "SNOW", "TMAX", "TMIN"

In [1]:
import os
import pandas as pd
import gc

In [27]:
#From Process Historical Flight Data
stations = ['USW00025704', 'USC00500275', 'US1AKBC0003', 'USW00027502', 'USW00026615', 'USW00026410', 'USW00027406', 'USC00502457', 'USC00501557', 'USW00025322', 'USR0000AJUN', 'USW00025325', 'USC00506166', 'USW00025501', 'USW00026616', 'USW00026617', 'USW00025329', 'USW00025333', 'USC00508107', 'USC00502587', 'USC00509919', 'USW00025339', 'US1ALJF0034', 'USW00013839', 'US1CHARM166', 'US1ALMB0042', 'USC00015547', 'USW00053922', 'USW00013964', 'US1ARPS0073', 'USC00034839', 'AQW00061705', 'USC00021050', 'USW00003103', 'USW00003195', 'USW00023183', 'USW00023160', 'USW00024283', 'USW00023155', 'USW00023152', 'US1CABT0009', 'USW00023254', 'USW00024286', 'USW00093193', 'USW00003144', 'USC00044278', 'USW00023129', 'USW00023174', 'USC00045280', 'USW00023258', 'USW00023259', 'USW00023230', 'USW00003102', 'USW00093110', 'USW00093138', 'USW00023182', 'USR0000CREA', 'USW00093225', 'USW00023188', 'USW00023234', 'USW00023293', 'USW00093206', 'USW00093184', 'USW00023190', 'USW00023273', 'USW00023213', 'USW00093230', 'USW00023237', 'US1CATL0005', 'USW00093073', 'USC00051777', 'USW00003017', 'USW00093005', 'USW00023063', 'US1COME0044', 'US1COGN0027', 'USW00093013', 'USW00093058', 'USC00058204', 'USW00014758', 'USW00014740', 'USW00013743', 'USW00093738', 'USW00013781', 'USW00012834', 'USW00012849', 'USW00012894', 'USW00012816', 'USW00013889', 'US1FLMN0016', 'USW00012896', 'USW00012838', 'USC00083909', 'USW00012897', 'USW00012815', 'USW00073805', 'USW00013899', 'USW00012812', 'USW00012871', 'USW00012873', 'USC00088756', 'USW00012842', 'USW00013858', 'USW00012844', 'USW00013869', 'US1GAFT0037', 'USW00003820', 'USW00093836', 'USC00092159', 'USW00003813', 'USW00013853', 'USW00093845', 'GQW00041415', 'USW00021503', 'USW00022521', 'USW00022516', 'USW00021510', 'USW00022534', 'USC00515275', 'USW00022536', 'US1IALN0004', 'US1IAWR0015', 'USC00133032', 'USW00014943', 'USW00094910', 'USC00101024', 'USC00103940', 'USW00024145', 'USW00024149', 'USW00024156', 'USW00094178', 'US1ILMCL018', 'USW00094870', 'USW00014819', 'US1ILCK0046', 'USW00094822', 'US1ILRI0018', 'US1ILSG0024', 'US1ILSG0020', 'USC00124300', 'USW00014827', 'US1INMR0008', 'USW00014848', 'USW00023064', 'US1KSEL0001', 'USW00003936', 'USW00013920', 'US1KSSG0076', 'USW00093814', 'USW00093820', 'USW00093821', 'USC00156130', 'USW00013934', 'US1LAEB0046', 'USW00013976', 'USW00003937', 'USW00013942', 'US1MSPK0002', 'US1LACD0002', 'USW00014739', 'USW00094720', 'USW00014756', 'USW00094724', 'USW00094746', 'US1MDAA0035', 'USW00014606', 'USW00014764', 'USW00094849', 'USW00014822', 'USC00208725', 'USW00014824', 'USC00205629', 'US1MIKN0053', 'USW00014858', 'USW00094893', 'USW00094815', 'US1MIIH0020', 'US1MIMG0013', 'USW00014841', 'USW00014845', 'USW00094824', 'USW00014850', 'USR0000MBEM', 'USW00094938', 'US1MNSL0003', 'USW00094931', 'CA006022487', 'US1MNHN0085', 'US1MNOL0011', 'USW00014926', 'USW00003945', 'USW00013987', 'USW00013988', 'US1MOPT0006', 'US1MOGR0087', 'US1MOFSA189', 'USW00013825', 'USC00229537', 'USW00013939', 'USC00225644', 'USC00226640', 'USW00003940', 'US1MSLD0010', 'USW00093862', 'USW00024033', 'USW00024132', 'USW00024135', 'USW00024143', 'USR0000MSET', 'USC00244560', 'USW00024153', 'USR0000MHEB', 'USW00003812', 'US1NCMK0087', 'USW00093740', 'USW00013723', 'US1NCCD0008', 'US1NCON0067', 'US1NCLR0030', 'USW00093719', 'US1NCMR0012', 'USW00013722', 'USW00013748', 'US1NDBH0027', 'USC00322148', 'USW00024012', 'US1NDCS0031', 'USW00094925', 'USW00014916', 'USW00014919', 'USW00094011', 'USW00024013', 'USW00094014', 'USW00014935', 'USW00014904', 'USW00024023', 'USW00014942', 'US10scot023', 'USW00014710', 'USW00013753', 'USW00014734', 'USW00014792', 'USW00023050', 'USW00023090', 'USC00294028', 'US1NMCH0015', 'USW00023049', 'USW00024121', 'USW00023169', 'USW00023185', 'USW00014735', 'USW00004725', 'USW00014733', 'USW00014748', 'USC00304174', 'USW00094789', 'USC00302868', 'USW00004781', 'USW00014714', 'USW00004724', 'USC00306659', 'USW00014768', 'USW00014771', 'USW00094790', 'USW00094745', 'USC00335750', 'USW00014820', 'USC00331777', 'USW00093815', 'USW00094830', 'USW00003950', 'USW00013967', 'USW00013968', 'USW00024221', 'USW00024224', 'USW00024225', 'USW00024284', 'USW00024229', 'USC00357050', 'USW00024232', 'USW00014737', 'USW00014860', 'USW00014711', 'USC00365259', 'USC00366880', 'US1PAAL0012', 'USW00014777', 'RQW00011603', 'RQC00666083', 'RQC00667297', 'RQW00011641', 'USW00014765', 'USW00013880', 'USW00013883', 'USW00013744', 'USW00003870', 'USW00013717', 'USC00390022', 'USW00024025', 'USW00024006', 'USW00024090', 'US1SDMH0077', 'USW00013877', 'USW00013882', 'USW00013891', 'US1MSCR0004', 'USC00406403', 'USW00013962', 'USW00023047', 'USW00013904', 'USW00012917', 'US1TXCMR043', 'USW00003904', 'USC00412011', 'USW00013960', 'USW00053914', 'US1TXTN0044', 'USW00022010', 'USW00023044', 'USW00003902', 'USW00012904', 'US1TXHRR023', 'USW00012960', 'USW00012918', 'USC00414792', 'US1TXWB0037', 'USW00003901', 'US1TXLK0022', 'US1TXHDL018', 'USW00023023', 'USC00417944', 'US1TXBXR143', 'USW00013972', 'USC00419367', 'USW00013959', 'USW00013966', 'USW00093129', 'USW00093075', 'USW00024126', 'USC00427061', 'USW00024127', 'USW00093198', 'USW00094030', 'USW00024193', 'USW00093736', 'USW00013733', 'USW00093741', 'USC00442368', 'USW00013740', 'USW00013741', 'US1VAAG0009', 'VQW00011640', 'VQW00011624', 'USW00014742', 'USW00024217', 'USW00024110', 'USW00024163', 'USW00024234', 'USW00024233', 'USW00024114', 'USW00024157', 'USW00024243', 'US1WIWN0001', 'USW00014991', 'US1WIBN0015', 'USC00474379', 'USW00094811', 'USC00475484', 'USC00475364', 'USW00004803', 'USW00013866', 'USW00003802', 'USW00003860', 'USC00465224', 'USW00024089', 'USW00024018', 'US1WYPK0034', 'USW00094023', 'USC00486428', 'USW00024022', 'USW00024027', 'USC00230887', 'USW00003177', 'USC00053867', 'US1SCBF0052', 'US1MIMQ0016', 'USW00023104', 'CQC00914801', 'CQC00914855', 'US1FLSJ0023', 'US1PACN0002', 'USC00229155', 'FMC00914429', 'USW00023195']

In [12]:
f = open("Historical Weather data/Data/2017.csv", 'r')
f.readline()

'US1MTMH0019,20170101,PRCP,43,,,N,\n'

In [59]:
#Preprocess the station data from the raw file
for year in range(2005, 2018):
    header = ['ID', 'DATE', 'TYPE','VALUE', 'TIME1', 'TIME2', "EXTRA", "EXTRA1"]
    raw = pd.read_csv("Historical Weather data/Data/" +str(year) + ".csv", header = None, names = header)
    raw = raw.drop(['TIME1', 'TIME2', "EXTRA", "EXTRA1"], axis = 1)
    gc.collect() #Run out of memory without manual garbage collection.
    raw = raw[raw['ID'].isin(stations)]
    gc.collect()
    raw = raw[raw['TYPE'].isin(['PRCP', 'TMAX', 'TMIN', 'SNOW'])]
    gc.collect()
    raw = raw.pivot_table(index = ['ID', 'DATE'], columns = 'TYPE')
    gc.collect()
    raw.to_csv("Historical Weather data/Data/" + str(year) + "_processed.csv")
    gc.collect()

In [26]:
#Combine all csvs into 1 file
names = ['ID', 'Date', 'PRCP', 'SNOW', 'TMAX', 'TMIN']
weather_data = pd.DataFrame(columns = names)
for fn in os.listdir('Historical Weather data\\Data\\'):
    filename = os.fsdecode(fn)
    if filename.endswith('.csv'):
        weather_data = weather_data.append(pd.read_csv('Historical Weather data/Data/' + filename, names = names, header= None, skiprows = 3))

In [28]:
weather_data.to_csv("Processed_Historical_Weather_Data.csv", index = False)